In [1]:
pip install nlpaug transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.8 MB/s eta 0:00:0000:0100:01
  Attempting uninst

In [2]:
import nlpaug.augmenter.word as naw
import nlpaug.model.word_stats as nmw
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

2025-07-29 19:12:20.025779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753816340.352573      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753816340.443644      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [3]:
import numpy as np 
import pandas as pd 

nice = pd.read_csv('/kaggle/input/final-dataset-with-stop-words/final_train_with_stopwords.csv')
nice['label'].value_counts()

label
not_applicable    3611
hope              1845
hate              1300
Name: count, dtype: int64

In [4]:
nice.duplicated().sum()

128

In [5]:
nice['text'].duplicated().sum()

202

In [6]:
nice.drop_duplicates(subset='text',inplace=True)
nice['label'].value_counts()

label
not_applicable    3517
hope              1774
hate              1263
Name: count, dtype: int64

In [7]:
hope_count = 3517 - 1774
hate_count = 3517 - 1263

In [ ]:
augmented = []

import nlpaug.augmenter.word as naw
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load the tokenizer and model for your Arabic classifier
model_name = 'aubmindlab/bert-base-arabertv2'  # or 'bert-large' if you want
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

label2id = {'hope': 0, 'hate': 1, 'not_applicable': 2,}
id2label = {v: k for k, v in label2id.items()}

model.config.label2id = label2id
model.config.id2label = id2label

# Create HF pipeline for text classification
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
# Initialize Arabic-compatible augmenter
augmenter = naw.ContextualWordEmbsAug(
    model_path='aubmindlab/bert-base-arabertv2',  # Arabic BERT model
    action="substitute",  # Substitute words
    aug_p=0.3  # Probability of augmenting a word
)

def augment_text_by_label(text, label):
    try:
        augmented_texts = augmenter.augment(text)
        if augmented_texts:
            augmented_text = augmented_texts[0]
            print("\n=== Augmentation Check ===")
            print(f"Original : {text}")
            print(f"Augmented: {augmented_text}")
            print("Changes:")
            print(highlight_changes(text, augmented_text))
            return augmented_text, label
        else:
            return text, label
    except Exception as e:
        print(f"Augmentation failed for text: {text}. Error: {e}")
        return text, label

for index, row in nice.iterrows():
    if row['label'] == 'hope':
        original_text = row['text']
        augmented_text, new_labels = augment_text_by_label(original_text, row['label'])

        pred_original = classifier(original_text)[0]['label']
        pred_augmented = classifier(augmented_text)[0]['label']

        # Map LABEL_0 → real label names if needed
        if pred_original.startswith("LABEL_"):
            pred_original = id2label[int(pred_original.split("_")[1])]
        if pred_augmented.startswith("LABEL_"):
            pred_augmented = id2label[int(pred_augmented.split("_")[1])]

        # Debug
        print(f"\nOriginal: {original_text}")
        print(f"Augmented: {augmented_text}")
        print(f"Prediction Original: {pred_original}")
        print(f"Prediction Augmented: {pred_augmented}")
        print(f"Match: {pred_original == pred_augmented}")
        print(f"Changed? {augmented_text != original_text}")

        if pred_original == pred_augmented and augmented_text != original_text:
            augmented.append({'text': augmented_text, 'label': new_labels})
            hope_count -= 1
            print("✅ Augmented sample accepted.")
        else:
            print("⚠️ Semantic drift detected – Skipping.")

        if hope_count == 0:
            break


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
The following layers were not sharded: bert.encoder.layer.*.attention.self.key.bias, bert.embeddings.position_embeddings.weight, cls.predictions.decoder.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.output.dense.weight, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.weight, cls.predictions.bias, cls.predictions.transform.dense.weight, bert.encoder.layer.*.attention.self.query.bias, cls.predictions.transform.LayerNorm.bias, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.attention.output.den


=== Augmentation Check ===
Original : اثق في قدرتي علي التعامل مع الصعوبات
Augmented: اثق ال+ قدرتي كل +ها مع حماس
Changes:
Augmentation failed for text: اثق في قدرتي علي التعامل مع الصعوبات. Error: name 'highlight_changes' is not defined

Original: اثق في قدرتي علي التعامل مع الصعوبات
Augmented: اثق في قدرتي علي التعامل مع الصعوبات
Prediction Original: hope
Prediction Augmented: hope
Match: True
Changed? False
⚠️ Semantic drift detected – Skipping.

=== Augmentation Check ===
Original : سلاما ياراحل عني القلب معك والروح معك وانا وراك اتبعك
Augmented: سلاما من عني القلب ل+ والروح ال+ سلام +ا يا
Changes:
Augmentation failed for text: سلاما ياراحل عني القلب معك والروح معك وانا وراك اتبعك. Error: name 'highlight_changes' is not defined

Original: سلاما ياراحل عني القلب معك والروح معك وانا وراك اتبعك
Augmented: سلاما ياراحل عني القلب معك والروح معك وانا وراك اتبعك
Prediction Original: hope
Prediction Augmented: hope
Match: True
Changed? False
⚠️ Semantic drift detected – Skipping.

=== Au

In [8]:
augmented_df = pd.DataFrame(augmented)
combined_df = pd.concat([nice, augmented_df], ignore_index=True)
combined_df.duplicated().sum()

128

In [9]:
combined_df['label'].value_counts()

label
not_applicable    3611
hope              3611
hate              1300
Name: count, dtype: int64

In [10]:
combined_df['text'].isnull().any()

False

In [11]:
combined_df['text'].duplicated().sum()

202

In [15]:
combined_df.drop_duplicates(inplace=True)
combined_df['label'].value_counts()

label
hope              3565
not_applicable    3552
hate              1277
Name: count, dtype: int64

In [14]:
combined_df['text'].duplicated().sum()

202

In [16]:
augmented_hate = []
def augment_text_by_hate_label(text, label):
    try:
        augmented_texts = augmenter.augment(text, n=2)  # Request two augmentations
        if len(augmented_texts) >= 2:
            return augmented_texts[0], augmented_texts[1], label
        elif len(augmented_texts) == 1:
            return augmented_texts[0], None, label
        else:
            return text, None, label
    except Exception as e:
        print(f"Augmentation failed for text: {text}. Error: {e}")
        return text, None, label

for index, row in nice.iterrows():
    if row['label'] == 'hate':
        augmented_text1, augmented_text2, new_labels = augment_text_by_hate_label(row['text'], row['label'])
        if augmented_text1 != row['text']:
            augmented_hate.append({'text': augmented_text1, 'label':new_labels})
            hate_count -= 1
        if hate_count == 0:
            break
        if augmented_text2 != None and augmented_text2 != row['text']:
            augmented_hate.append({'text': augmented_text2, 'label':new_labels})
            hate_count -= 1
        if hate_count == 0:
            break

In [17]:
len(augmented_hate)

2311

In [18]:
augmented_df1 = pd.DataFrame(augmented_hate)
combined_df1 = pd.concat([nice, augmented_df1], ignore_index=True)
combined_df1.duplicated().sum()

128

In [19]:
print(combined_df.duplicated().sum(),combined_df1.duplicated().sum())

0 128


In [20]:
combined_df1.drop_duplicates(inplace=True)

In [21]:
len(augmented)

1766

In [22]:
combined = pd.concat([nice, augmented_df1,augmented_df], ignore_index=True)
combined.duplicated().sum()

128

In [23]:
pd.merge(augmented_df, nice, on=['text'])

,text,label_x,label_y


In [24]:
pd.merge(augmented_df1, nice, on=['text', 'label'])

,text,label


In [25]:
print(nice.duplicated().sum(),augmented_df.duplicated().sum(),augmented_df1.duplicated().sum())

128 0 0


In [26]:
combined.drop_duplicates()

,text,label
0,و لانني احب الاشياء الراقيه,not_applicable
1,اثق في قدرتي علي التعامل مع الصعوبات,hope
2,وروضه بات طل الغيث ينسجها حتي اذا نسجت اضحي يد...,not_applicable
3,اشعر بقلبي العنيد وهو يحارب الله في بع,not_applicable
4,بتحسسني اني مرتضي ده كان بيدي واحد محتاج صدقه ...,not_applicable
...,...,...
10828,وسط هذه الحياه و+ سمع +نا +كم +ون الله يسخر لن...,hope
10829,محمد مصطفى الله ما من ادمي عملا انجي له من نور...,hope
10830,وتعيش يا شيخ [رابط],hope
10831,البهجه 19 2 »,hope


In [27]:
combined['label'].value_counts()

label
not_applicable    3611
hope              3611
hate              3611
Name: count, dtype: int64

In [28]:
augmented_df.to_csv('augmented_hope.csv', index=False)
augmented_df1.to_csv('augmented_hate.csv', index=False)
combined.to_csv('combined.csv', index=False)

In [29]:
dev = pd.read_csv('/kaggle/input/final-dataset-with-stop-words/final_dev_with_stopwords.csv')
dev['label'].value_counts()

label
not_applicable    780
hope              395
hate              260
Name: count, dtype: int64

In [30]:
dev.duplicated().sum()

10

In [31]:
dev.drop_duplicates(inplace =True)
dev['label'].value_counts()

label
not_applicable    775
hope              390
hate              260
Name: count, dtype: int64

In [32]:
hope_count = 775 - 390
hate_count = 775 - 260

In [33]:
augmented = []

def augment_text_by_label(text, label):
    try:
        augmented_texts = augmenter.augment(text)
        return augmented_texts[0] if augmented_texts else text, label
    except Exception as e:
        print(f"Augmentation failed for text: {text}. Error: {e}")
        return text, label

for index, row in dev.iterrows():
    if row['label'] == 'hope':
        augmented_text, new_labels = augment_text_by_label(row['text'], row['label'])
        if augmented_text != row['text']:
            augmented.append({'text': augmented_text, 'label':new_labels})
            hope_count -= 1
        if hope_count == 0:
            break 

In [34]:
augmented_dev = pd.DataFrame(augmented)
combined_dev = pd.concat([dev, augmented_dev], ignore_index=True)
combined_dev.duplicated().sum()

0

In [35]:
combined_dev.drop_duplicates(inplace=True)
combined_dev['label'].value_counts()

label
hope              775
not_applicable    775
hate              260
Name: count, dtype: int64

In [38]:
augmented_hate = []
def augment_text_by_hate_label(text, label):
    try:
        augmented_texts = augmenter.augment(text, n=2)  # Request two augmentations
        if len(augmented_texts) >= 2:
            return augmented_texts[0], augmented_texts[1], label
        elif len(augmented_texts) == 1:
            return augmented_texts[0], None, label
        else:
            return text, None, label
    except Exception as e:
        print(f"Augmentation failed for text: {text}. Error: {e}")
        return text, None, label

for index, row in dev.iterrows():
    if row['label'] == 'hate':
        augmented_text1, augmented_text2, new_labels = augment_text_by_hate_label(row['text'], row['label'])
        if augmented_text1 != row['text']:
            augmented_hate.append({'text': augmented_text1, 'label':new_labels})
            hate_count -= 1
        if hate_count == 0:
            break
        if augmented_text2 != None and augmented_text2 != row['text']:
            augmented_hate.append({'text': augmented_text2, 'label':new_labels})
            hate_count -= 1
        if hate_count == 0:
            break

In [39]:
len(augmented_hate)

515

In [40]:
augmented_dev1 = pd.DataFrame(augmented_hate)
combined_dev1 = pd.concat([dev, augmented_dev1], ignore_index=True)
combined_dev1.duplicated().sum()

0

In [41]:
combined_dev1['label'].value_counts()

label
not_applicable    775
hate              775
hope              390
Name: count, dtype: int64

In [42]:
combined = pd.concat([dev, augmented_dev1,augmented_dev], ignore_index=True)
combined.to_csv('combined_dev.csv', index=False)